In [1]:
# |default_exp rag

## Install dependencies

## Make an app with Gradio

In [2]:
from datetime import datetime

# |export
import ollama
import re
import gradio as gr
from concurrent.futures import ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama import OllamaEmbeddings
from chromadb.config import Settings
from chromadb import Client
import chromadb
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os
import csv
from google import genai
from google.genai import types
from fastcore.net import urljson, HTTPError
from openai import api_key
from openai import OpenAI
import openai

In [3]:
# |export
import textwrap
import numpy as np
from datetime import datetime
import pandas as pd
from chromadb import Documents, EmbeddingFunction, Embeddings
from IPython.display import Markdown
# import chromadb.utils.embedding_functions as embedding_functions
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [4]:
load_dotenv()
# deepseek_key = os.getenv('DEEPSEEK_R1_bAPI_KEY')
gemini_key = os.getenv('GEMINI_API_KEY')

In [7]:
print(os.environ.get('HTTPS_PROXY'))

http://127.0.0.1:20171


In [8]:
import requests
requests.get("https://google.com")

<Response [200]>

In [9]:
client = genai.Client(api_key=gemini_key)

In [10]:
all_models = client.models.list()
for m in all_models.page:
    if 'embedContent' in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [11]:
#| export
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = "models/text-embedding-004"
        # model = "models/gemini-embedding-exp-03-07"
        # model = "models/text-embedding-001"
        title = "Custom query"
        result = client.models.embed_content(model=model,
                                   contents=input,
                                   config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY'),
                                   # config=types.EmbedContentConfig(task_type='RETRIEVAL_DOCUMENT'),
                                   )
        return result.embeddings[0].values

In [12]:
'p'+str(1)

'p1'

In [13]:
#| export
def create_chroma_db(documents, name):
    chroma_client = chromadb.PersistentClient(path="../db")
    # chroma_client = chromadb.Client()
    coll = chroma_client.get_or_create_collection(name=name,
                                                embedding_function=GeminiEmbeddingFunction(),
                                                  metadata={
                                                      "description": "Employee manual",
                                                      "created_by": "binjian",
                                                      "created": str(datetime.now())
                                                  })
    # coll.add(
    #     documents=[d.page_content for d in documents],
    #     metadatas=[d.metadata for d in documents],
    #     ids=['p'+str(i+1) for i in range(len(documents))]
    # )
    # return coll
    for i,d  in enumerate(documents):
         try:
             coll.add(
                 documents=d.page_content,
                 metadatas=d.metadata,
                 ids=str(i+1)
             )
             print(f"Added document {i+1}")
         except Exception as e:
             print(f"{i+1},{e}")

    return coll

In [14]:
#| e# xport
loader = PyMuPDFLoader("../res/DeepSeek_R1.pdf")
documents = loader.load()
docs = [d.page_content for d in documents]

In [15]:
result = client.models.embed_content(#model="models/gemini-embedding-exp-03-07",
                                     model="models/text-embedding-004",
                                     contents=documents[0].page_content,
                                     config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY')
                                     )

In [17]:
result.embeddings[0].values

[-0.005321432,
 0.0023961603,
 -0.054761603,
 0.014361784,
 0.022462582,
 0.04692675,
 0.021321893,
 0.013300336,
 -0.017185612,
 -0.06399107,
 0.0034897667,
 0.07250505,
 0.032994296,
 0.029214822,
 0.056890476,
 -0.0035143262,
 0.087437496,
 0.068535805,
 -0.045277454,
 0.01164426,
 -0.0400441,
 -0.015364517,
 -0.009231976,
 -0.042674735,
 -0.047258932,
 -0.060211927,
 0.0677147,
 -0.015525728,
 0.020592438,
 -0.048398323,
 0.02014073,
 0.029914249,
 0.0340472,
 -0.07474625,
 0.03429146,
 0.020654717,
 0.0020258583,
 -0.012901867,
 -0.019215176,
 -0.025971308,
 -0.02988938,
 0.03155073,
 -0.004431291,
 0.0058627157,
 -0.06494266,
 -0.056483135,
 -0.016261045,
 -0.01741983,
 -0.025090039,
 0.019287035,
 0.020605098,
 -0.05946599,
 -0.034993414,
 -0.009929958,
 -0.027896129,
 -0.054743182,
 -0.059155043,
 -0.015893215,
 0.06430048,
 0.0014203138,
 -0.03364598,
 -0.045411274,
 -0.026156532,
 -0.01872012,
 0.0024419674,
 -0.02921977,
 -0.062408578,
 0.019189037,
 -0.019649548,
 -0.038738

In [18]:
#|export
db = create_chroma_db(documents, "deepseek_r1_retrieval")
# chroma_client = chromadb.PersistentClient(path="../db")
# db = chroma_client.get_or_create_collection('deepseek_r1')


Added document 1
Added document 2
Added document 3
Added document 4
Added document 5
Added document 6
Added document 7
Added document 8
Added document 9
Added document 10
Added document 11
Added document 12
Added document 13
Added document 14
Added document 15
Added document 16
Added document 17
Added document 18
Added document 19
Added document 20
Added document 21
Added document 22


In [19]:
# chroma_client = chromadb.Client()
recs = db.peek(5)
# df = pd.DataFrame(recs)

In [20]:
def get_relevant_passage(query, db):
    passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
    return passage

In [24]:
# Perform embedding search
passage = get_relevant_passage("How does the distilled models perform in evaluation comparing to other models?", db)
Markdown(passage)

DeepSeek-R1 avoids introducing length bias during GPT-based evaluations, further solidifying
its robustness across multiple tasks.
On math tasks, DeepSeek-R1 demonstrates performance on par with OpenAI-o1-1217,
surpassing other models by a large margin. A similar trend is observed on coding algorithm
tasks, such as LiveCodeBench and Codeforces, where reasoning-focused models dominate these
benchmarks. On engineering-oriented coding tasks, OpenAI-o1-1217 outperforms DeepSeek-R1
on Aider but achieves comparable performance on SWE Verified. We believe the engineering
performance of DeepSeek-R1 will improve in the next version, as the amount of related RL
training data currently remains very limited.
3.2. Distilled Model Evaluation
Model
AIME 2024
MATH-500
GPQA
LiveCode
CodeForces
Diamond
Bench
pass@1
cons@64
pass@1
pass@1
pass@1
rating
GPT-4o-0513
9.3
13.4
74.6
49.9
32.9
759
Claude-3.5-Sonnet-1022
16.0
26.7
78.3
65.0
38.9
717
OpenAI-o1-mini
63.6
80.0
90.0
60.0
53.8
1820
QwQ-32B-Preview
50.0
60.0
90.6
54.5
41.9
1316
DeepSeek-R1-Distill-Qwen-1.5B
28.9
52.7
83.9
33.8
16.9
954
DeepSeek-R1-Distill-Qwen-7B
55.5
83.3
92.8
49.1
37.6
1189
DeepSeek-R1-Distill-Qwen-14B
69.7
80.0
93.9
59.1
53.1
1481
DeepSeek-R1-Distill-Qwen-32B
72.6
83.3
94.3
62.1
57.2
1691
DeepSeek-R1-Distill-Llama-8B
50.4
80.0
89.1
49.0
39.6
1205
DeepSeek-R1-Distill-Llama-70B
70.0
86.7
94.5
65.2
57.5
1633
Table 5 | Comparison of DeepSeek-R1 distilled models and other comparable models on
reasoning-related benchmarks.
As shown in Table 5, simply distilling DeepSeek-R1’s outputs enables the efficient DeepSeek-
R1-7B (i.e., DeepSeek-R1-Distill-Qwen-7B, abbreviated similarly below) to outperform non-
reasoning models like GPT-4o-0513 across the board. DeepSeek-R1-14B surpasses QwQ-32B-
Preview on all evaluation metrics, while DeepSeek-R1-32B and DeepSeek-R1-70B significantly
exceed o1-mini on most benchmarks. These results demonstrate the strong potential of distilla-
tion. Additionally, we found that applying RL to these distilled models yields significant further
gains. We believe this warrants further exploration and therefore present only the results of the
simple SFT-distilled models here.
4. Discussion
4.1. Distillation v.s. Reinforcement Learning
In Section 3.2, we can see that by distilling DeepSeek-R1, the small model can achieve impressive
results. However, there is still one question left: can the model achieve comparable performance
through the large-scale RL training discussed in the paper without distillation?
To answer this question, we conduct large-scale RL training on Qwen-32B-Base using math,
code, and STEM data, training for over 10K steps, resulting in DeepSeek-R1-Zero-Qwen-32B. The
experimental results, shown in Table 6, demonstrate that the 32B base model, after large-scale
14

In [25]:
def make_prompt(query, relevant_passage):
    escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
    prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
    Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
    However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
    strike a friendly and converstional tone. \
    If the passage is irrelevant to the answer, you may ignore it.
    QUESTION: '{query}'
    PASSAGE: '{relevant_passage}'

        ANSWER:
    """).format(query=query, relevant_passage=escaped)
    return prompt

In [26]:
query ="How does the distilled models perform in evaluation comparing to other models?"
prompt = make_prompt(query, passage)
Markdown(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.     Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.     However, you are talking to a non-technical audience, so be sure to break down complicated concepts and     strike a friendly and converstional tone.     If the passage is irrelevant to the answer, you may ignore it.
    QUESTION: 'How does the distilled models perform in evaluation comparing to other models?'
    PASSAGE: 'DeepSeek-R1 avoids introducing length bias during GPT-based evaluations, further solidifying its robustness across multiple tasks. On math tasks, DeepSeek-R1 demonstrates performance on par with OpenAI-o1-1217, surpassing other models by a large margin. A similar trend is observed on coding algorithm tasks, such as LiveCodeBench and Codeforces, where reasoning-focused models dominate these benchmarks. On engineering-oriented coding tasks, OpenAI-o1-1217 outperforms DeepSeek-R1 on Aider but achieves comparable performance on SWE Verified. We believe the engineering performance of DeepSeek-R1 will improve in the next version, as the amount of related RL training data currently remains very limited. 3.2. Distilled Model Evaluation Model AIME 2024 MATH-500 GPQA LiveCode CodeForces Diamond Bench pass@1 cons@64 pass@1 pass@1 pass@1 rating GPT-4o-0513 9.3 13.4 74.6 49.9 32.9 759 Claude-3.5-Sonnet-1022 16.0 26.7 78.3 65.0 38.9 717 OpenAI-o1-mini 63.6 80.0 90.0 60.0 53.8 1820 QwQ-32B-Preview 50.0 60.0 90.6 54.5 41.9 1316 DeepSeek-R1-Distill-Qwen-1.5B 28.9 52.7 83.9 33.8 16.9 954 DeepSeek-R1-Distill-Qwen-7B 55.5 83.3 92.8 49.1 37.6 1189 DeepSeek-R1-Distill-Qwen-14B 69.7 80.0 93.9 59.1 53.1 1481 DeepSeek-R1-Distill-Qwen-32B 72.6 83.3 94.3 62.1 57.2 1691 DeepSeek-R1-Distill-Llama-8B 50.4 80.0 89.1 49.0 39.6 1205 DeepSeek-R1-Distill-Llama-70B 70.0 86.7 94.5 65.2 57.5 1633 Table 5 | Comparison of DeepSeek-R1 distilled models and other comparable models on reasoning-related benchmarks. As shown in Table 5, simply distilling DeepSeek-R1’s outputs enables the efficient DeepSeek- R1-7B (i.e., DeepSeek-R1-Distill-Qwen-7B, abbreviated similarly below) to outperform non- reasoning models like GPT-4o-0513 across the board. DeepSeek-R1-14B surpasses QwQ-32B- Preview on all evaluation metrics, while DeepSeek-R1-32B and DeepSeek-R1-70B significantly exceed o1-mini on most benchmarks. These results demonstrate the strong potential of distilla- tion. Additionally, we found that applying RL to these distilled models yields significant further gains. We believe this warrants further exploration and therefore present only the results of the simple SFT-distilled models here. 4. Discussion 4.1. Distillation v.s. Reinforcement Learning In Section 3.2, we can see that by distilling DeepSeek-R1, the small model can achieve impressive results. However, there is still one question left: can the model achieve comparable performance through the large-scale RL training discussed in the paper without distillation? To answer this question, we conduct large-scale RL training on Qwen-32B-Base using math, code, and STEM data, training for over 10K steps, resulting in DeepSeek-R1-Zero-Qwen-32B. The experimental results, shown in Table 6, demonstrate that the 32B base model, after large-scale 14'

        ANSWER:
    

In [32]:
# client = genai.Client(api_key=gemini_key)
# response = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
response = client.models.generate_content(model="gemini-2.5-pro-exp-03-25", contents=prompt)
Markdown(response.text)

Okay, let's break down how those "distilled models" stack up against others, based on the information provided!

Think of "distillation" here like taking the knowledge from a very big, powerful model (DeepSeek-R1) and transferring it into smaller, more efficient models. The passage shows that this process works quite well!

Here's how the distilled versions of DeepSeek-R1 compare:

1.  **Even Smaller Distilled Models Shine:** The relatively small 7-billion parameter version (called DeepSeek-R1-Distill-Qwen-7B) actually manages to outperform a well-known model like GPT-4o-0513 across all the reasoning-related tests listed in their table (Table 5).
2.  **Mid-Sized Distilled Models Surpass Competitors:** The 14-billion parameter distilled model (DeepSeek-R1-Distill-Qwen-14B) performs better than another model called QwQ-32B-Preview on all the evaluation scores shown.
3.  **Larger Distilled Models Lead Significantly:** The bigger distilled versions, specifically the 32-billion and 70-billion parameter ones (DeepSeek-R1-Distill-Qwen-32B and DeepSeek-R1-Distill-Llama-70B), significantly beat OpenAI's o1-mini model on most of the benchmark tests.

So, in a nutshell, the evaluation shows that these distilled models perform very impressively, often outperforming other well-known models, even when the distilled models are smaller in size. This highlights that distillation is a really effective technique for creating powerful but efficient models!

In [31]:
Markdown(response.text)

Alright, let's break down how the distilled DeepSeek-R1 models perform compared to others! Basically, by using a technique called "distillation" on DeepSeek-R1, they've created smaller, more efficient versions. These smaller models, like the DeepSeek-R1-Distill-Qwen-7B, actually outperform other models that aren't focused on reasoning, such as GPT-4o-0513, across the board. Furthermore, the DeepSeek-R1-14B model does better than the QwQ-32B-Preview on all the tests, and the DeepSeek-R1-32B and DeepSeek-R1-70B models are much better than o1-mini on most of the benchmarks. So, distillation seems to be a pretty powerful tool!


In [25]:
# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
# embedding = client.embeddings.create(
#         model="deepseek/deepseek-r1:free",
#         input='Your text string goes here'
#     )
# embedding

In [26]:
# Parallelize embedding generation
def generate_embedding(chunk):
    # return ollama_embedding_function.embed_query(chunk.page_content)
    return google_ef.embed_query(chunk.page_content)


with ThreadPoolExecutor() as executor:
    embeddings = list(executor.map(generate_embedding, chunks))

AttributeError: 'OpenAIEmbeddingFunction' object has no attribute 'embed_query'

In [35]:

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "binjian.github.io", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "binjian's digital garden", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="deepseek/deepseek-r1:free",
  messages=[
    {
      "role": "system", "content": "You are a helpful assistant.",
      "role": "user", "content": "What's Anthropic's Model context protocol?"
    }
  ]
)
print(completion.choices[0].message.content)

Anthropic's Model Context Protocol refers to the structured approach used in their AI models, particularly Claude, to manage conversational context effectively while prioritizing safety and coherence. Here's a breakdown of its key components:

1. **Context Window Management**: 
   - Claude's large context window (up to 200k tokens) allows processing lengthy inputs, but the protocol ensures efficient retention of relevant information. Techniques like attention mechanisms prioritize recent or critical dialogue segments while avoiding overload.

2. **Safety and Alignment**:
   - Rooted in **Constitutional AI**, the protocol incorporates ethical guidelines to filter harmful requests, reduce biases, and ensure outputs align with human values. It may refuse unsafe tasks or seek clarification for ambiguous queries.

3. **Multi-Turn Coherence**:
   - Maintains consistency across long conversations by tracking dialogue history, user intent, and key entities. This avoids repetitive or contradict

In [40]:
google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=gemini_key)

In [46]:
def convert_qa_to_csv(input_file, output_file):
    """
    Convert a text file with Q/A format to a CSV file.

    Args:
        input_file: Path to the input text file
        output_file: Path to the output CSV file
    """
    # Read the content of the file
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the content by 'Q' marker
    qa_blocks = content.split('Q\n')

    qa_blks = [block.strip() for block in qa_blocks][1:]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [[line for line in block.split('\n') ] for block in qa_blks if block.strip()]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [blk for block in qa_blocks if block.strip() for blk in block.strip()]

    # Process each Q&A block
    qa_pairs = []
    for block in qa_blks:
        # Split the block into lines
        lines = block.strip().split('\n')

        if lines:
            # First line is the question
            question = lines[0]
            # The rest are the answer
            answer = '\n'.join(lines[1:])

            # Add the pair to our list
            qa_pairs.append([question, answer])

    # Write to CSV
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Write header
        writer.writerow(['Question', 'Answer'])
        # Write Q&A pairs
        for pair in qa_pairs:
            writer.writerow(pair)
    print(f"Conversion complete. CSV file saved to {output_file}")

In [47]:

input_files = ["../res/qa_service.txt", "../res/qa_technology.txt"]
output_files = ["../res/qa_service.csv", "../res/qa_technology.csv"]
for in_f, ot_f in zip(input_files, output_files):
    convert_qa_to_csv(in_f, ot_f)
    

Conversion complete. CSV file saved to ../res/qa_service.csv
Conversion complete. CSV file saved to ../res/qa_technology.csv


In [48]:
# |export
client = chromadb.PersistentClient(path="../vdb")
# collections = [client.get_or_create_collection(name="siasun_qa_service",embedding_function=deepseek_ef),
#                 client.get_or_create_collection(name="siasun_qa_technology",embedding_function=deepseek_ef)]
collections = [client.get_or_create_collection(name="siasun_qa_service", embedding_function=google_ef),
                client.get_or_create_collection(name="siasun_qa_technology", embedding_function=google_ef)]

In [11]:
i=2
f'q{i}'

'q2'

In [49]:
# |export
for csv_file,collection in zip(output_files,collections):
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            collection.add(
                documents = row,
                metadatas = [{"source": "question"}, {"source": "answer"}],
                ids = [f"{collection.name}_q{i}", f"{collection.name}_a{i}"]
            )

TypeError: RetryError.__init__() missing 1 required positional argument: 'cause'

In [19]:
#|export
queries=["你们的产品需要多久维护一次?","我怎么设置机器人的安全工作区域?"]
queries[0]

'你们的产品需要多久维护一次?'

In [10]:
results = collections[0].query(
    query_texts=queries,
    n_results=4
)

In [11]:
results

{'ids': [['siasun_qa_service_q2',
   'siasun_qa_service_a9',
   'siasun_qa_service_q21',
   'siasun_qa_service_a10'],
  ['siasun_qa_service_q1',
   'siasun_qa_service_a19',
   'siasun_qa_service_q22',
   'siasun_qa_service_a18']],
 'embeddings': None,
 'documents': [['你们的产品，多久需要维护一次？维护保养内容有哪些？',
   '作业作为一个重要的单元，所以不能进行批量删除与添加，防止误操作造成损失。',
   '宏作业是干什么用的？',
   '新松有标准的视觉通讯协议，视觉厂家可以按照此协议进行开发适配。当前适配过的品牌有，沈阳自动化所，欧姆龙、海康、梅卡曼德、视比特、基恩士。'],
  ['我们的人员调试不熟练，进度慢，你们厂家能负责调试吗？',
   '机器人打精度用到的开关。',
   '机器人可以存储多少个作业？',
   '零位设定，是机器人在零位时的码盘值，码盘输入、码盘输出是以零位时为零开始计数的码盘值。']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}],
  [{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}]],
 'distances': [[0.35835238473584624,
   0.8026765812953612,
   0.8398664268040978,
   0.8905950951966901],
  [0.5583867931983068,
   0.5964258746641342,
   0.6352793698255077,
   0.73287700

In [12]:
# results['metadatas'][0] #[0]['source']
results['documents'][0][0]

'你们的产品，多久需要维护一次？维护保养内容有哪些？'

In [13]:
colls = client.list_collections()
colls[0]

'siasun_qa_service'

In [47]:
queries1 = ['你们的产品需要多久维护一次?','宏作业有什么用?']

In [48]:

results = collections[0].query(
    query_texts=queries1,
    n_results=4
)
results


{'ids': [['siasun_qa_service_q2',
   'siasun_qa_service_a9',
   'siasun_qa_service_q21',
   'siasun_qa_service_a10'],
  ['siasun_qa_service_q16',
   'siasun_qa_service_a20',
   'siasun_qa_service_q13',
   'siasun_qa_service_a7']],
 'embeddings': None,
 'documents': [['你们的产品，多久需要维护一次？维护保养内容有哪些？',
   '作业作为一个重要的单元，所以不能进行批量删除与添加，防止误操作造成损失。',
   '宏作业是干什么用的？',
   '新松有标准的视觉通讯协议，视觉厂家可以按照此协议进行开发适配。当前适配过的品牌有，沈阳自动化所，欧姆龙、海康、梅卡曼德、视比特、基恩士。'],
  ['离线建模使用是样册的标准杆长，但实际机器人内有杆长补偿，这样建模对离线下发的点位准确性是否有影响？',
   '有，离线接口库。',
   '欧拉角是什么？有什么用处？新松机器人的欧拉角顺序是什么？',
   '还有其它报警跟随，还需要查看其它报警来确认原因。弧焊与离线都会产生离线运动失败的报警。']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}],
  [{'source': 'question'},
   {'source': 'answer'},
   {'source': 'question'},
   {'source': 'answer'}]],
 'distances': [[0.35835238473584624,
   0.8026765812953612,
   0.8398664268040978,
   0.8905950951966901],
  [0.5112318120344393,
   0.5778327751539375,
  

In [14]:

# question = "你们的产品需要多久维护一次?"
# question = "你们在售前评估上，如何帮助到我们?"
answers = []
for collection in collections:
    results = collection.query(
        query_texts=queries,
        n_results=4
    )
    docs = []
    for i,metadata in enumerate(results['metadatas'][0]):
        if metadata['source'] == 'question':
            docs.append({'id': results['ids'][0][i],
                         'document': results['documents'][0][i],
                         'distance':results['distances'][0][i]})
    df = pd.DataFrame(docs)
    answers.append(df)
df_answers = pd.concat(answers, axis=0,ignore_index=True)
# df_answers = pd.stack(answers, axis=2)

In [15]:
df_answers.loc[df_answers['distance'].idxmin()]

id          siasun_qa_technology_q10
document         你们在售前评估上，能提供什么样的帮助？
distance                    0.376322
Name: 2, dtype: object

In [16]:

id_q = df_answers.loc[df_answers['distance'].idxmin()]['id']
id_q

'siasun_qa_technology_q10'

In [17]:

id_a_list = id_q.split('_')
id_a_list[-1] = id_a_list[-1].replace('q','a')
id_a_list

['siasun', 'qa', 'technology', 'a10']

In [18]:
id_a = '_'.join(id_a_list)
id_a

'siasun_qa_technology_a10'

In [19]:
coll_idx = 0 if id_a_list[-2] == 'service' else 1
coll_idx

1

In [20]:
answer = collections[coll_idx].get(id_a)
answer['documents']

['部分情况可提供现场技术指导，提供成功应用案例经验支持，仿真模拟场景，评估负载等风险，提供机械、电气、软件接口对接。']

In [21]:
best_answer = df_answers.loc[df_answers['distance'].idxmin()]

In [42]:
def qa(questions:list[str], collections:list[chromadb.Collection]=collections):
    matched_questions = []
    for collection in collections:
        results = collection.query(
            query_texts=questions,
            n_results=4
        )
        docs = []
        for i,metadata in enumerate(results['metadatas'][0]):
            if metadata['source'] == 'question':
                docs.append({'id': results['ids'][0][i],
                                'document': results['documents'][0][i], 
                                'distance':results['distances'][0][i]})
        df = pd.DataFrame(docs)
        matched_questions.append(df)

    df_matched_questions = pd.concat(matched_questions,axis=0,ignore_index=True)
    best_match_q_id = df_matched_questions.loc[df_matched_questions['distance'].idxmin()]['id']
    id_a_list = best_match_q_id.split('_')
    id_a_list[-1] = id_a_list[-1].replace('q','a')
    id_a = '_'.join(id_a_list)
    coll_idx = 0 if id_a_list[-2] == 'service' else 1
    best_answer = collections[coll_idx].get(id_a)['documents']
    res_text = best_answer[0]
    return res_text
    # question =
    # return answers

In [34]:

question = "你们的产品需要多久维护一次?"
res = qa(question)
res


['根据机器人的型号和实际使用情况，制定机器人的保养计划,一般分为日常、3 个月、6 个月、1 年期的维护保养。\n需要对机器人进行日常点检和定期维护保养，点检工作主要检查设备是否存在漏油、异响、异常震动、异常报警；定期维护保养主要对油脂、线束护套、风扇、电机接头等易损位置进行检查，并定期更换润滑油。具体检验项目及维护周期详见安装维护手册。']

In [36]:
print(res[0])

根据机器人的型号和实际使用情况，制定机器人的保养计划,一般分为日常、3 个月、6 个月、1 年期的维护保养。
需要对机器人进行日常点检和定期维护保养，点检工作主要检查设备是否存在漏油、异响、异常震动、异常报警；定期维护保养主要对油脂、线束护套、风扇、电机接头等易损位置进行检查，并定期更换润滑油。具体检验项目及维护周期详见安装维护手册。


In [43]:
# |export
iface = gr.Interface(fn=qa, inputs=gr.Text(value="多久维护一次产品?"), outputs="text")
iface.launch(width=500,share=False)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [41]:
# this is only necessary in a notebook
iface.close()

Closing server running on port: 7860


## Create a `requirements.txt` file

In [26]:
%%writefile ../requirements.txt
fastcore

Overwriting ../requirements.txt


In [27]:
# | hide
import nbdev

nbdev.nbdev_export()

In [28]:
# |default_exp data_preprocessing

## Convert this notebook into a Gradio app

In [29]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [30]:
# | hide
import nbdev

nbdev.nbdev_export()